In [1]:
# jesseliu2000
import torch
import torch.nn as nn
import torch.optim as optim


class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.embedding = nn.Embedding(n_words, embedding_size)
        self.fc1 = nn.Linear(n_token * embedding_size, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.fc3 = nn.Linear(n_hidden2, n_words)
        self.tanh = nn.Tanh()

    def forward(self, X):
        X1 = self.embedding(X) # [batch_size, seq_length, embedding_size]
        X1 = X1.view(-1, n_token * embedding_size) # [batch_size, seq_length * embedding_size]
        X2 = self.fc1(X1) # [batch_size, hidden_size]
        X_act = self.tanh(X2) # [batch_size, hidden_size]
        X2 = self.tanh(self.fc2(X_act))
        output = self.fc3(X2)
        # print(output.size())
        return output



if __name__ == '__main__':
    n_token = 2  # number of token in one sentence
    n_hidden1 = 4  # number of hidden size
    n_hidden2 = 2  # number of hidden size
    embedding_size = 2

    file_list = ['I hate milk', 'Its cold today', 'I like cat']

    word_vocab = ' '.join(file_list).split()
    word_vocab = list(set(word_vocab))
    word2index = {w: i for i, w in enumerate(word_vocab)}
    index2word = {i: w for i, w in enumerate(word_vocab)}

    n_words = len(word2index)  # number of whole word

    input = [i.split(' ')[:-1] for i in file_list]
    map_input = [[word2index[i] for i in j] for j in input]
    target = [i.split(' ')[-1] for i in file_list]
    map_target = [word2index[i] for i in target]


    # promise the same input length
    map_input = torch.LongTensor(map_input)
    map_target = torch.LongTensor(map_target)

    model = NNLM()

    lr = 1e-3
    epoch = 5000
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training
    for ep in range(epoch):
        optimizer.zero_grad()
        output = model(map_input)
        loss = criterion(output, map_target)

        if (ep + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Test
    predict = model(map_input).data.max(1, keepdim=True)[1]
    predict = predict.squeeze()
    print("Input:", [sentence.split()[:2] for sentence in file_list])
    print("OutPut:", [index2word[n.item()] for n in predict])

Epoch: 1000 cost = 0.622854
Epoch: 2000 cost = 0.420115
Epoch: 3000 cost = 0.088562
Epoch: 4000 cost = 0.027916
Epoch: 5000 cost = 0.013745
Input: [['I', 'hate'], ['Its', 'cold'], ['I', 'like']]
OutPut: ['milk', 'today', 'cat']
